![alt text](https://pictures.s3.yandex.net/resources/S04_04_1682601335.png)

Полный список и описания CBV модуля `django.contrib.auth` есть [в документации](https://docs.djangoproject.com/en/5.1/topics/auth/default/#module-django.contrib.auth.views).

***
## Подключение urls модуля django.contrib.auth

Файл *urls.py* модуля `django.contrib.auth` выглядит так:

```py
# django/contrib/auth/urls.py
from django.contrib.auth import views
from django.urls import path


urlpatterns = [
    # Логин.
    path('login/', views.LoginView.as_view(), name='login'), 
    # Логаут.
    path('logout/', views.LogoutView.as_view(), name='logout'), 

    # Изменение пароля.
    path('password_change/', views.PasswordChangeView.as_view(), name='password_change'),
    # Сообщение об успешном изменении пароля.
    path('password_change/done/', views.PasswordChangeDoneView.as_view(), name='password_change_done'),

    # Восстановление пароля.
    path('password_reset/', views.PasswordResetView.as_view(), name='password_reset'),
    # Сообщение об отправке ссылки для восстановления пароля.
    path('password_reset/done/', views.PasswordResetDoneView.as_view(), name='password_reset_done'),
    # Вход по ссылке для восстановления пароля.
    path('reset/<uidb64>/<token>/', views.PasswordResetConfirmView.as_view(), name='password_reset_confirm'),
    # Сообщение об успешном восстановлении пароля.
    path('reset/done/', views.PasswordResetCompleteView.as_view(), name='password_reset_complete'),
] 
```

Этот файл можно подключить к проекту как обычный urls.py пользовательского приложения. 

Чтобы все запросы к адресам `django.contrib.auth` обрабатывались маршрутизатором этого модуля — добавим к адресу префикс *auth/* (можно задать и любой другой префикс: *accounts/*, *user/*, какой угодно ещё — и всё будет работать).

Подключите файл к головному *urls.py* проекта:

```py
# acme_project/urls.py
...

urlpatterns = [
    ...
    # Подключаем urls.py приложения для работы с пользователями.
    path('auth/', include('django.contrib.auth.urls')),
    ...
] 
```

Запустите проект и проверьте доступность страницы восстановления пароля: http://127.0.0.1:8000/auth/password_reset/

***
## Кастомизация шаблонов

Шаблоны для страниц модуля `django.contrib.auth` хранятся не в самом модуле, а в  `django.contrib.admin`, в директории *django/contrib/admin/templates/registration/*. Это связано с тем, что эти шаблоны используются и в админке, для тех же сценариев. Дизайн шаблонов выдержан в стиле админ-зоны Django. 

Но при входе или при работе с паролем пользователи должны видеть страницы в стиле основного сайта; значит, необходимо подменить встроенные шаблоны на собственные. Изменять встроенные шаблоны нельзя: эти изменения будут доступны только на компьютере разработчика; при развёртывании проекта на любом другом компьютере фреймворк Django будет установлен в исходном виде и с исходными шаблонами.

Итак, требуется:

* создать собственные шаблоны (их можно сделать по образцу встроенных шаблонов);

* сохранить шаблоны в проекте;
указать Django, откуда брать шаблоны для CBV модуля `django.contrib.auth`.

Создать набор шаблонов и сохранить их где-нибудь в коде проекта — не проблема: создаём в корне проекта директорию для шаблонов и копируем туда оригинальные файлы шаблонов из *django/contrib/admin/templates/registration/*. Потом можно будет изменить HTML-код этих шаблонов так, чтобы они выглядели в стиле остальных страниц сайта.

Остаётся объяснить фреймворку, что шаблоны нужно брать именно из этой директории.

Сделать это можно двумя способами:

1. Для каждого CBV в файле *acme_projects/urls.py* указать путь в `urlpatterns` и в явном виде задать имя шаблона. Например, для страницы *logout/* путь будет описан так:

    ```py
    urlpatterns = [
        ...
        path(
            'logout/',
            views.LogoutView.as_view(template_name='logged_out.html'),
            name='logout',
        ),
        ...
    ] 
    ```
    
    Так нужно будет описать восемь маршрутов, в которых для каждого CBV будет указано имя нового шаблона. Код станет громоздким; сейчас вместо этих восьми маршрутов применяется лишь одна короткая строка:

    ```py
    path('auth/', include('django.contrib.auth.urls')),
    ```

2. Каждому CBV по умолчанию назначен свой HTML-шаблон, имя шаблона можно посмотреть [в документации](https://docs.djangoproject.com/en/5.1/topics/auth/default/#all-authentication-views) или в таблице в начале урока. Например, шаблон для CBV `LogoutView` называется *logged_out.html*. 

    Все эти шаблоны лежат в директории *registration/*; CBV ищут свои шаблоны по адресам *registration/<имя_шаблона*>. 

    Шаблоны в Django могут храниться в различных местах, и, когда вызван какой-то шаблон, первым делом Django начинает его искать в каталоге *templates/* корневой директории; если не находит — идёт искать по директориям приложений.

    Значит, если в корневом каталоге *templates/* создать собственную директорию *registration/* и в ней расположить шаблоны с заданными именами, то Django будет использовать именно их: при обнаружении первого же совпадающего имени шаблона Django берёт найденный шаблон и прекращает поиск.

В каталоге *templates/*, расположенном в корне проекта, создайте директорию *registration/*, а в ней создайте пустые HTML-файлы для шаблонов.

А можно просто скопировать в *templates/* каталог *registration/* вместе со всеми файлами из *django/contrib/admin/templates/*, кроме файла *password_reset_email*.html — это шаблон для письма о сбросе пароля, его менять не нужно.

Обратите внимание: в приложении `django.contrib.admin` в каталоге с шаблонами нет файла *login.html* — а он нам тоже понадобится. Этот файл всё равно придётся создать вручную.

***
## Кастомный шаблон для страницы входа: LoginView

Как и во всех CBV модулях `django.contrib.auth`, класс `LoginView` применяет шаблон по умолчанию, этот шаблон должен называться *login.html*. 

Однако шаблона с таким именем в директории *django/contrib/admin/templates/registration* нет: разработчик должен создать его самостоятельно. По мнению авторов Django, страницы логина могут сильно различаться в разных проектах, поэтому делать единый для всех шаблон не имеет смысла.

В документации приведён вариант HTML-кода для шаблона *login.html*: 

```html
{% extends "base.html" %}

{% block content %}

<!-- Этот блок мы уберём, в нашей форме ошибки выводятся внутри формы. -->
{% if form.errors %}
<p>Your username and password didn't match. Please try again.</p>
{% endif %}

{% if next %}
    {% if user.is_authenticated %}
    <p>Your account doesn't have access to this page. To proceed,
    please login with an account that has access.</p>
    {% else %}
    <p>Please login to see this page.</p>
    {% endif %}
{% endif %}

<form method="post" action="{% url 'login' %}">
{% csrf_token %}
<table>
<tr>
    <td>{{ form.username.label_tag }}</td>
    <td>{{ form.username }}</td>
</tr>
<tr>
    <td>{{ form.password.label_tag }}</td>
    <td>{{ form.password }}</td>
</tr>
</table>

<input type="submit" value="login">
<!-- Этот тег надо будет перенести в нашу форму. -->
<input type="hidden" name="next" value="{{ next }}">
</form>

{# Assumes you setup the password_reset view in your URLconf #}
<p><a href="{% url 'password_reset' %}">Lost password?</a></p>

{% endblock %} 
```

Адаптируем код под стиль сайта, заодно переведём сообщения на русский язык. 

Скопируйте в файл *login.html* код из листинга:

```html
<!-- templates/registration/login.html -->
{% extends "base.html" %}
<!-- Загружаем библиотеку для работы со стилями Bootstrap. -->
{% load django_bootstrap5 %}

{% block content %}
  <!-- Если в запросе передан GET-параметр с указанием страницы, 
    куда надо перейти после входа. -->
  {% if next %}
    <!-- Если пользователь уже залогинен, но не обладает нужными правами. -->
    {% if user.is_authenticated %}
      <p>
        У вашего аккаунта нет доступа к этой странице.
        Чтобы продолжить, войдите в систему с аккаунтом,
        у которого есть доступ.
      </p>
    {% else %}
      <p>
        Пожалуйста, войдите в систему,
        чтобы просматривать эту страницу.
      </p>
    {% endif %}
  {% endif %}

  <div class="card col-4 m-3">
    <div class="card-header">
      Войти в систему
    </div>
    <div class="card-body">
      <!-- В атрибуте action указываем адрес, куда должен отправляться запрос. -->
      <form method="post" action="{% url 'login' %}">
        {% csrf_token %}
        {% bootstrap_form form %}
        <!-- В скрытом поле передаём параметр next, 
          это URL для переадресации после логина. -->
        <input type="hidden" name="next" value="{{ next }}">
        {% bootstrap_button button_type="submit" content="Войти" %}
      </form>
      <div>
        <!-- Ссылка для перехода на страницу восстановления пароля. -->
        <a href="{% url 'password_reset' %}">Забыли пароль?</a>
      </div>
    </div>
  </div>
{% endblock %} 
```

После успешного входа сайт по умолчанию переадресует пользователя на страницу accounts/profile/, но в нашем проекте такой страницы нет. 

Адрес для редиректа можно изменить в настройках проекта: в константе `LOGIN_REDIRECT_URL` можно указать либо относительный путь (например `/birthday/`), либо имя URL-паттерна (`namespace` и `name` пути страницы в urls.py; например `pages:homepage`). 

После логина будем перенаправлять пользователя на главную страницу — `homepage`. Добавьте в файл с настройками константу `LOGIN_REDIRECT_URL`:

```py
# acme_project/settings.py
...

LOGIN_REDIRECT_URL = 'pages:homepage' 
```

Откройте страницу http://127.0.0.1:8000/auth/login/ и попробуйте залогиниться ещё раз. Теперь никаких ошибок быть не должно.

***
## Кастомный шаблон страницы выхода: LogoutView

Страницей выхода из аккаунта управляет CBV `LogoutView`. Его шаблон — *logged_out.html* (название шаблона есть в [документации](https://docs.djangoproject.com/en/5.1/topics/auth/default/#django.contrib.auth.views.LogoutView)); если этот шаблон ещё не создан — создайте его и скопируйте в него код из листинга:

```html
{% extends "base.html" %}

{% block content %}
  <h2>Вы вышли из системы!</h2>
{% endblock %} 
```

При отправке POST-запроса на адрес http://127.0.0.1:8000/auth/logout/ пользователь разлогинивается, а на экране видит сообщение:

***
## Навигация для анонимного и залогиненного пользователя

Чтобы пользователю было удобно пользоваться страницами логина и логаута, добавим в навигационную панель ссылки на эти страницы. Залогиненному пользователю должна быть видна кнопка для выхода из системы, а анонимному пользователю — ссылка на форму входа. Также выведем в навигацию username залогиненного пользователя.

Для начала просто добавим в *header.html* все ссылки, которые должны быть в навигации, вне зависимости от того, для какого пользователя они предназначены. Для разлогивания необходимо выполнять POST-запрос. Кликом на ссылку этого сделать нельзя, но можно сделать с помощью формы.

```html
<!-- templates/includes/header.html -->
<header>
  <nav class="navbar shadow-sm">
    <div class="container">
      {% with request.resolver_match.view_name as view_name %}
        <ul class="nav nav-pills">
          <li class="nav-item">
            <a class="nav-link {% if view_name == 'pages:homepage' %} active {% endif %}"
              href="{% url 'pages:homepage' %}">
              Главная
            </a>
          </li>
          <li class="nav-item">
            <a class="nav-link {% if view_name == 'birthday:create' %} active {% endif %}"
              href="{% url 'birthday:create' %}">
              Калькулятор дней рождения
            </a>
          </li>
          <li class="nav-item">
            <a class="nav-link {% if view_name == 'birthday:list' %} active {% endif %}" href="{% url 'birthday:list' %}">
              Cписок дней рождения
            </a>
          </li>

          <!-- Это блок ссылок для авторизованного пользователя -->
          <span class="navbar-text">Пользователь: <b>{{ user.username }}</b></span>
          <li class="nav-item">
            <form method="post" action="{% url 'logout' %}" >
              {% csrf_token %}
              <button type="submit" class="nav-link">Выйти</button>
            </form>
          </li>
          <!-- Конец блока ссылок для авторизованного пользователя -->

          <!-- А здесь блок ссылок для неавторизованного пользователя -->
          <li class="nav-item">
            <a class="nav-link{% if view_name == 'login' %} active {% endif %}" href="{% url 'login' %}">Войти</a>
          </li>        
          <!-- Конец блока ссылок для неавторизованного пользователя -->        
        </ul>
      {% endwith %}
    </div>
  </nav>
</header> 
```

Теперь нужно описать условие: «если пользователь авторизован — показывать первый блок ссылок; если пользователь не авторизован — показывать второй блок».

Прямо в шаблоне можно определить, какой пользователь отправил запрос к странице: в словаре контекста под ключом `user` передаётся объект пользователя, сделавшего запрос. Обратившись к атрибутам этого объекта, можно узнать много полезного. 

По названиям полей можно получить свойства объекта — например, для авторизованного пользователя можно получить `user.email`, `user.is_superuser` и другие свойства; можно определить, авторизован ли пользователь: свойство `user.is_authenticated` возвращает `True` или `False`.
